In [1]:
import sys
sys.path.append('..')

In [2]:
from nbhd import data

In [3]:
db = data.Base()

Initializing database connection...
Database connected!


In [4]:
# db.query('''ALTER TABLE roads
#          RENAME COLUMN "roadFunction" to road_function;''')

In [5]:
db.info('roads')

,Column,Datatype
0,id,text
1,endNode,text
2,startNode,text
3,roadNumberTOID,text
4,roadNameTOID,text
5,fictitious,boolean
6,roadClassification,text
7,road_function,text
8,formOfWay,text
9,length,bigint


In [6]:
road_counts = db.count('roads', 'road_function')

In [7]:
road_sig = {'Motorway': 0, 'A Road':1, 'B Road':2, 'Minor Road':3,
           'Local Road':4, 'Local Access Road':5, 'Restricted Local Access Road':6,
           'Secondary Access Road':7}

In [8]:
road_counts.reset_index(inplace=True)

In [9]:
road_counts['significance'] = road_counts['index'].apply(lambda x: road_sig[x])

In [10]:
road_counts = road_counts.sort_values('significance')

In [11]:
road_counts = road_counts.append([{'index':'Total', 'counts': road_counts.counts.sum()}])

In [12]:
road_counts.index = range(9)

In [13]:
road_counts

,index,counts,significance
0,Motorway,7124,0.0
1,A Road,285536,1.0
2,B Road,159415,2.0
3,Minor Road,676167,3.0
4,Local Road,1650781,4.0
5,Local Access Road,45307,5.0
6,Restricted Local Access Road,803087,6.0
7,Secondary Access Road,133522,7.0
8,Total,3760939,NaN


In [14]:
quartile_length = dict()
for road_type in road_counts['index'].values:
    print(road_type)
    quartile_length[road_type] = dict()
    for q in (0.25, 0.5, 0.75):
        quartile_length[road_type][q] = int(db.percentile('roads', 'length', 'road_function', 
                                                          road_type, q))

Motorway
A Road
B Road
Minor Road
Local Road
Local Access Road
Restricted Local Access Road
Secondary Access Road
Total


In [15]:
import pandas as pd

In [16]:
pd.DataFrame(quartile_length).T

,0.25,0.50,0.75
Motorway,85,327,570
A Road,33,69,169
B Road,39,85,204
Minor Road,43,101,275
Local Road,43,68,109
Local Access Road,44,73,137
Restricted Local Access Road,50,100,200
Secondary Access Road,39,53,86
Total,43,76,146


In [17]:
road_counts.set_index('index',inplace=True)

In [18]:
road_counts.drop(columns=['significance'],inplace=True)

In [42]:
df = pd.concat([road_counts, pd.DataFrame(quartile_length).T], axis=1)

In [45]:
total = df.counts['Total']

In [46]:
df['Percentage'] = (df.counts/total * 1000).astype(int)/10

In [57]:
df['Count (%)'] = df.apply(lambda x: f'{"{:,}".format(x.counts)} ({str(x.Percentage)})' , axis=1)

In [23]:
df['IQR'] = df[0.75] - df[0.5]

In [55]:
df = df[['counts', 'Count (%)','Percentage',0.25,0.5,0.75,'IQR']]

KeyError: "['IQR'] not in index"

In [25]:
df = df.drop(columns=['counts','Percentage'])

In [26]:
df0 = df.T

In [27]:
df0

,Motorway,A Road,B Road,Minor Road,Local Road,Local Access Road,Restricted Local Access Road,Secondary Access Road,Total
Count (%),"7,124.0 (0.1)","285,536.0 (7.5)","159,415.0 (4.2)","676,167.0 (17.9)","1,650,781.0 (43.8)","45,307.0 (1.2)","803,087.0 (21.3)","133,522.0 (3.5)","3,760,939.0 (100.0)"
0.25,85,33,39,43,43,44,50,39,43
0.5,327,69,85,101,68,73,100,53,76
0.75,570,169,204,275,109,137,200,86,146
IQR,243,100,119,174,41,64,100,33,70


In [48]:
df.index.name = 'Road Function'

In [58]:
df[['Count (%)']].T

Road Function,Motorway,A Road,B Road,Minor Road,Local Road,Local Access Road,Restricted Local Access Road,Secondary Access Road,Total
Count (%),"7,124 (0.1)","285,536 (7.5)","159,415 (4.2)","676,167 (17.9)","1,650,781 (43.8)","45,307 (1.2)","803,087 (21.3)","133,522 (3.5)","3,760,939 (100.0)"


In [29]:
t = 'Percentile Length (m)'
mi = pd.MultiIndex.from_tuples(
[
 ('Count (%)',''),
 (t, 0.25),
 (t, 0.5),
 (t, 0.75),
 (t, 'IQR')]
)

In [30]:
df00 = pd.DataFrame(df0.values,index=mi)

In [31]:
df00.columns = df0.columns

In [32]:
df00.T

Count (%) Percentile Length (m)       \
                                                                   0.25  0.5   
Road Function                                                                  
Motorway                            7,124.0 (0.1)                    85  327   
A Road                            285,536.0 (7.5)                    33   69   
B Road                            159,415.0 (4.2)                    39   85   
Minor Road                       676,167.0 (17.9)                    43  101   
Local Road                     1,650,781.0 (43.8)                    43   68   
Local Access Road                  45,307.0 (1.2)                    44   73   
Restricted Local Access Road     803,087.0 (21.3)                    50  100   
Secondary Access Road             133,522.0 (3.5)                    39   53   
Total                         3,760,939.0 (100.0)                    43   76   

                                        
                             0.75  IQR  
Road Function                           
Motorway                      570  243  
A Road                        169  100  
B Road                        204  119  
Minor Road                    275  174  
Local Road                    109   41  
Local Access Road             137   64  
Restricted Local Access Road  200  100  
Secondary Access Road          86   33  
Total                         146   70

In [33]:
import os

In [39]:
with open(os.path.join('../../csv','road_function.md'),'w') as f:
    f.write(df00.T.to_markdown())

In [38]:
df00.T.to_csv('../../csv/road_function.csv')

In [36]:
df00

Road Function                    Motorway           A Road           B Road  \
Count (%)                   7,124.0 (0.1)  285,536.0 (7.5)  159,415.0 (4.2)   
Percentile Length (m) 0.25             85               33               39   
                      0.5             327               69               85   
                      0.75            570              169              204   
                      IQR             243              100              119   

Road Function                     Minor Road          Local Road  \
Count (%)                   676,167.0 (17.9)  1,650,781.0 (43.8)   
Percentile Length (m) 0.25                43                  43   
                      0.5                101                  68   
                      0.75               275                 109   
                      IQR                174                  41   

Road Function              Local Access Road Restricted Local Access Road  \
Count (%)                     45,307.0 (1.2)             803,087.0 (21.3)   
Percentile Length (m) 0.25                44                           50   
                      0.5                 73                          100   
                      0.75               137                          200   
                      IQR                 64                          100   

Road Function              Secondary Access Road                Total  
Count (%)                        133,522.0 (3.5)  3,760,939.0 (100.0)  
Percentile Length (m) 0.25                    39                   43  
                      0.5                     53                   76  
                      0.75                    86                  146  
                      IQR                     33                   70

In [37]:
df00.T

Count (%) Percentile Length (m)       \
                                                                   0.25  0.5   
Road Function                                                                  
Motorway                            7,124.0 (0.1)                    85  327   
A Road                            285,536.0 (7.5)                    33   69   
B Road                            159,415.0 (4.2)                    39   85   
Minor Road                       676,167.0 (17.9)                    43  101   
Local Road                     1,650,781.0 (43.8)                    43   68   
Local Access Road                  45,307.0 (1.2)                    44   73   
Restricted Local Access Road     803,087.0 (21.3)                    50  100   
Secondary Access Road             133,522.0 (3.5)                    39   53   
Total                         3,760,939.0 (100.0)                    43   76   

                                        
                             0.75  IQR  
Road Function                           
Motorway                      570  243  
A Road                        169  100  
B Road                        204  119  
Minor Road                    275  174  
Local Road                    109   41  
Local Access Road             137   64  
Restricted Local Access Road  200  100  
Secondary Access Road          86   33  
Total                         146   70

In [35]:
local_roads =db.query('''
select split_part(name1, ' ', 2)  AS name_ending,
        geometry
       FROM roads WHERE road_function LIKE 'Local Road'
''', spatial=True)

In [38]:
name_counts = local_roads.value_counts('name_ending')

In [50]:
df = name_counts.loc[name_counts>1000].reset_index()

In [55]:
df.loc[df.name_ending!='']

,name_ending,0
0,Road,334653
1,Street,142496
2,Close,136917
3,Avenue,127178
4,Drive,107196
5,Way,72411
6,Lane,69680
8,Crescent,46360
9,Gardens,32574
10,Place,28331
